In [ ]:
from qiskit_algorithms import Grover
# from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
from qiskit_algorithms import AmplificationProblem
from qiskit import transpile, ClassicalRegister, QuantumRegister, qasm3
from qiskit.transpiler import CouplingMap
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
import gen_utilities as utl
importlib.reload(utl)
os.chdir(current_dir)
import math
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
from qiskit.visualization import plot_histogram
import json
import pickle
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
import qiskit
from qiskit.providers.options import Options
from qiskit_ibm_runtime.options.twirling_options import TwirlingOptions
from qiskit_ibm_runtime.options import SamplerOptions
from qiskit_ibm_runtime import SamplerV2
from qiskit import QuantumCircuit
from qiskit.converters import (
    circuit_to_dag, 
    dag_to_circuit
    )
from copy import deepcopy
from qiskit.circuit.library import TGate, HGate, SXGate, XGate, SGate
from qiskit.synthesis import SolovayKitaevDecomposition
from qiskit.synthesis import generate_basic_approximations
from qiskit.transpiler.passes import SolovayKitaev

print(qiskit.__version__)

# BASIS=["cx", "rz", "h"]
# BASIS=["cx", "h", "s", "t"]

SHOTS=200000

def meas_qubits(circ: QuantumCircuit, target_qubits):
    '''Meas the desired qubits. Gives the meas label.'''
    meas_cregs=ClassicalRegister(len(target_qubits), "meas")
    circ.add_register(meas_cregs)
    for idx, q in enumerate(target_qubits):
        circ.measure(q, meas_cregs[idx])

def create_noise_est_cir(circ):
    noise_est_circ=utl.replace_gates_h_sx_rx(circ)
    return noise_est_circ

def replace_t_with_injection(circ: QuantumCircuit, num_qubits: int):
    '''Replaces t gate with injection.'''
    dag=circuit_to_dag(circ)
    throwaway_cr=ClassicalRegister(1, "throw_away")
    meas_cr=ClassicalRegister(num_qubits, "meas")
    qr=QuantumRegister(num_qubits+1)
    new_qc=QuantumCircuit(qr, meas_cr, throwaway_cr)
    op_nodes=dag.op_nodes()
    for node in op_nodes:
        # print(node.name)
        if node.name=="t" or node.name=="tdg":
            new_qc.h(qr[num_qubits])
            if node.name=="t":
                new_qc.t(qr[num_qubits])
            else:
                new_qc.tdg(qr[num_qubits])
            new_qc.cx(node.qargs[0]._index, qr[num_qubits])
            if node.name=="t":
                new_qc.cs(num_qubits, node.qargs[0]._index)
            else:
                new_qc.csdg(num_qubits, node.qargs[0]._index)
            new_qc.measure(qr[num_qubits], throwaway_cr[0])
            new_qc.reset(qr[num_qubits])

        else:
            utl.copy_node(new_qc, node)
    return new_qc

def split_circuit(circuit, start, end):
    nq = len(circuit.qubits)
    qc2 = QuantumCircuit(nq)
    for x in circuit[start:end]:
        qc2.append(x[0], x[1])
    return qc2

def ghz(num_qubits):
    circ=QuantumCircuit(num_qubits)
    circ.h(0)
    for idx in list(range(num_qubits))[1::]:
        if idx not in list([3,5,7]):
            circ.cx(0, idx)
    # for idx in list(range(num_qubits))[:-1:]:
    #     circ.cx(idx, idx+1)
    return circ

def ghz_mid20():
    circ=QuantumCircuit(20)
    mid=10
    lower=list(range(10))[::-1]
    upper=list(range(11, 20))#[::-1]
    circ.h(mid)
    circ.cx(mid, lower[0])
    circ.cx(mid, upper[0])
    for qubit_group in [lower, upper]:
        for idx in range(len(qubit_group)-1):
            circ.cx(qubit_group[idx], qubit_group[idx+1])
    return circ

In [2]:
from qiskit.result import Result
from qiskit_ibm_runtime import QiskitRuntimeService

# service=QiskitRuntimeService()
# backend=service.backend("ibm_torino")
# target="1"*4
# num_qubits=len(target)
p1_error_prob=0.001
p2_error_prob=10*p1_error_prob
p1_depol=depolarizing_error(p1_error_prob, 1)
p2_depol=depolarizing_error(p2_error_prob, 2)
noise_model=NoiseModel()
noise_model.add_all_qubit_quantum_error(p1_depol, ["measure", "x", "rz", "sx", "reset"])
noise_model.add_all_qubit_quantum_error(p2_depol, ["cz"])

noisy_sim=AerSimulator(noise_model=noise_model)
# noisy_sim=AerSampler(options=dict(run_options=dict(noise_model=noise_model)))
service = QiskitRuntimeService()

# backend=service.backend("ibm_marrakesh")
# noisy_sim=SamplerV2(backend)

In [ ]:
# Ideal execution
BASIS=["cz", "x", "rz", "sx"]

num_qubits=20
# circ=ghz_mid20()
circ=qasm3.load("dicke_n20_k1.qasm")
# oracle = Statevector.from_label(target)
# problem = AmplificationProblem(oracle, is_good_state=target)
# # grover_op=problem.grover_operator
# # optimal_num_iterations = math.floor(
# #     math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**num_qubits)))
# # )
# optimal_num_iterations=1
# circ=Grover().construct_circuit(problem, optimal_num_iterations)
# basic_approx=generate_basic_approximations(basis_gates=[TGate(), HGate(), XGate(), SGate()], depth=3)
# skd=SolovayKitaev(recursion_degree=2, basic_approximations=basic_approx)
# print(circ.draw(fold=-1))
# skd=SolovayKitaev(recursion_degree=2)
# circ=skd(circ)
# circ.measure_all()
meas_qubits(circ, list(range(num_qubits)))
# circ=transpile(circ, basis_gates=["cx", "u"], optimization_level=3)
print(circ.draw(fold=-1))
ideal_circ=deepcopy(circ)

ideal_sim=AerSimulator()
result_ideal=ideal_sim.run(ideal_circ, shots=SHOTS).result()
ideal_counts=result_ideal.get_counts()
print(ideal_counts)
ideal_dist={k:v/SHOTS for k, v in ideal_counts.items()}


In [ ]:
# circ=transpile(circ, optimization_level=3, basis_gates=BASIS)
# circ=transpile(circ, initial_layout=list(range(num_qubits)), seed_transpiler=0, optimization_level=3, basis_gates=BASIS)
# print(circ.draw(fold=-1))
# circ=replace_t_with_injection(circ, num_qubits)
print(circ.draw(fold=-1))
circ=transpile(circ, seed_transpiler=0, optimization_level=3, basis_gates=BASIS)
circ=utl.remove_idle_wires(circ)
noise_est_circ=create_noise_est_cir(circ)
clifford_noise_est_cir=utl.circ_replace_rz(noise_est_circ, num_qubits)
clifford_noise_est_cir=utl.remove_idle_wires(clifford_noise_est_cir)
print(f"noise payload circ {circ.draw(fold=-1)}")
print(f"noise estimation circ {noise_est_circ.draw(fold=-1)}")
print("ops payload: ", circ.count_ops())
print("ops noise est cir: ", noise_est_circ.count_ops())
print("ops clifford noise est cir: ", clifford_noise_est_cir.count_ops())
      

In [ ]:
result_cliff_noise_est=ideal_sim.run(clifford_noise_est_cir, shots=SHOTS).result()
result_noisy=noisy_sim.run([circ, noise_est_circ], shots=SHOTS).result()


In [15]:
def strip_classical_reg(remove_pos: int, dist: dict):
    '''Modifies the keys of dist by splitting it the key by space and removing the strings at index remove_pos.
    This is from stripping unwanted classical registers.'''
    new_dict={}
    for k, v in dist.items():
        k=k.split(" ")
        k.pop(remove_pos)
        k="".join(k)
        count=new_dict.get(k, 0)
        count+=v
        new_dict[k]=count
    return new_dict

In [ ]:
# print(result_noisy)
# counts_ideal_noise_est=strip_classical_reg(0, result_cliff_noise_est.get_counts())
# noisy_counts=strip_classical_reg(0, result_noisy.get_counts()[0])
# counts_noisy_noise_est=strip_classical_reg(0, result_noisy.get_counts()[1])

counts_ideal_noise_est=result_cliff_noise_est.get_counts()
noisy_counts=result_noisy.get_counts()[0]
counts_noisy_noise_est=result_noisy.get_counts()[1]

print(noisy_counts)
print(counts_noisy_noise_est)
dist_noisy_noise_est={k:v/SHOTS for k,v in counts_noisy_noise_est.items()}
print(counts_ideal_noise_est)

In [17]:
with open("ideal_counts_sim.json", "w") as f:
    json.dump(ideal_counts, f)

with open("noisy_noise_est_dist_sim.json", "w") as f:
    json.dump(dist_noisy_noise_est, f)

with open("ideal_noise_est_counts_sim.json", "w") as f:
    json.dump(counts_ideal_noise_est, f)

with open("noisy_counts_sim.json", "w") as f:
    json.dump(noisy_counts, f)

In [ ]:
print(noise_model)